In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import gc
import time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from time import gmtime, strftime

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score,KFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from nltk.stem.porter import PorterStemmer
from sklearn.decomposition import TruncatedSVD

import scipy
import gc
import sys

#Add https://www.kaggle.com/anttip/wordbatch to your kernel Data Sources, 
#until Kaggle admins fix the wordbatch pip package installation
# sys.path.insert(0, '../input/wordbatch/wordbatch/')
##import wordbatch
##from wordbatch.extractors import WordBag, WordHash
##from wordbatch.models import FTRL, FM_FTRL

from nltk.corpus import stopwords
import re

def rmsle(y, y_pred):
     assert len(y) == len(y_pred)
     y = np.array(y)
     y_pred = np.array(y_pred)
     y_pred[y_pred<0] = 0
     y[y<0] = 0   
     
     return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_pred), 2)))
    

In [3]:
NUM_BRANDS = 4500
NUM_CATEGORIES = 1250

develop = False
# develop= True

start_time = time.time()
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2018-02-06 20:04:59


In [36]:
LOAD_TEST = True
MAKE_SAMPLE = False

start = time.clock() 
 
train = pd.read_table('input/train.tsv', engine='c')
if MAKE_SAMPLE:
    train = train.sample(n=100000).reset_index().drop('index',axis=1).copy(deep=True)
 

print('Train shape: ', train.shape)
merged = pd.concat([train])
nrow_train = train.shape[0]
del train


if LOAD_TEST:
    test = pd.read_table('input/test.tsv', engine='c')
    if MAKE_SAMPLE:
        test = test.sample(n=100000).reset_index().drop('index',axis=1).copy(deep=True)
    
    print('Test shape: ', test.shape)
    merged = pd.concat([merged, test])
    nrow_test = test.shape[0]
    del test
    
print('Merged shape: ', merged.shape)
print ("Time passed min:",(time.clock()-start)/60)

merged.head()

Train shape:  (1482535, 8)
Test shape:  (693359, 7)
Merged shape:  (2175894, 9)
Time passed min: 0.13181709575634007


,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id
0,NaN,Men/Tops/T-shirts,3,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,NaN,0.0
1,Razer,Electronics/Computers & Tablets/Components & P...,3,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,NaN,1.0
2,Target,Women/Tops & Blouses/Blouse,1,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,NaN,2.0
3,NaN,Home/Home Décor/Home Décor Accents,1,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,NaN,3.0
4,NaN,Women/Jewelry/Necklaces,1,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,NaN,4.0


In [37]:
## preprocess - splitting category names
##


start = time.clock()
tt = pd.DataFrame([ str(x).split("/") for x in merged['category_name']])
cat_col_names = ['general_cat','subcat_1','subcat_2','subcat_3','subcat_4']
tt.columns = ['general_cat','subcat_1','subcat_2','subcat_3','subcat_4']

merged[cat_col_names] = tt
merged["category_name"] = merged["category_name"].fillna("Other").astype("category")

## merged.drop('category_name', axis=1, inplace=True)
del tt
print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 0.18494980450585102


In [38]:
## replace missing values
start = time.clock()
for col_name in  cat_col_names+['brand_name']+['item_description']:
    merged['mis_'+col_name] = 0
    merged[col_name].fillna(value='missing', inplace=True)
    merged.loc[merged[col_name]=='missing','mis_'+col_name] = 1

merged.head()
print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 0.05808154993574798


In [39]:
VAL_MISS_MARGIN = 10

### delete rare brands
##
start = time.clock()
pop_brand = merged['brand_name'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
merged.loc[merged['brand_name'].isin(pop_brand), 'mis_brand_name'] = 1
merged.loc[merged['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'

## delete rare categories
##

pop_category1 = merged['general_cat'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category2 = merged['subcat_1'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
pop_category3 = merged['subcat_2'].value_counts().loc[lambda x: x<=VAL_MISS_MARGIN].index.values
# merged.loc[merged['general_cat'].isin(pop_brand), 'mis_general_cat'] = 1
# merged.loc[merged['general_cat'].isin(pop_category1), 'general_cat'] = 'missing'
# merged.loc[merged['subcat_1'].isin(pop_brand), 'mis_subcat_1'] = 1
# merged.loc[merged['subcat_1'].isin(pop_category2), 'subcat_1'] = 'missing'
# merged.loc[merged['subcat_2'].isin(pop_brand), 'mis_subcat_2'] = 1
# merged.loc[merged['subcat_2'].isin(pop_category3), 'subcat_2'] = 'missing'
    
print (pop_brand)
print (len(pop_category1),pop_category1[1:5])
print (len(pop_category2),pop_category2[1:5])
print (len(pop_category3),pop_category3[1:5])
print ("Time passed min:",(time.clock()-start)/60)

['Doe' 'Travis Mathew' 'ExerSaucer' ..., 'First Alert' 'Archie Comics'
 'David Meister']
0 []
0 []
154 ['Diaper Stackers & Caddies' 'Calendar' 'Kitchen Safety' 'Presentation']
Time passed min: 0.09121411741102557


In [40]:
## count non-utf characters
s = "☆Please read bio☆ Women's sandal bundle"
def count_nonutf(s):
    s_clean = re.sub(r'[^\x00-\x7F]+','', s)
    return len(s)-len(s_clean)

start = time.clock()
tt = pd.DataFrame([ count_nonutf(x) for x in merged['item_description']])
merged['non_utf_descr'] = tt

tt = pd.DataFrame([ count_nonutf(x) for x in merged['name']])
merged['non_utf_name'] = tt

print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 0.19306816159459003


In [11]:
## Building features with counts of all Non-utf symbols
##
s = "☆Please read bio☆ Women's sandal bundle"

def all_non_utf(s):
    s_clean = re.sub(r'[\x00-\x7F]','', s)
    char_dict = dict()
    for k in s_clean:
        if k not in char_dict.keys():
            char_dict[k]=s_clean.count(k)
    return char_dict

start = time.clock()
print ("Counting non utf symbols")
tt = pd.DataFrame([ all_non_utf(x) for x in merged['item_description']])
tt.fillna(0,inplace=True)

print ("Converting to sparse")
X_non_utf_uniq = scipy.sparse.csr_matrix(X_uniq)

print ("Doing svd")
svd = TruncatedSVD(n_components = 10)
X_non_utf_uniq = svd.fit_transform(X_non_utf_uniq)
X_non_utf_uniq = pd.DataFrame(X_non_utf_uniq)
print ("Time passed min:",(time.clock()-start)/60)

Counting non utf symbols


KeyboardInterrupt: 

In [342]:
ss = "!!sdfdsf"
s_clean = re.sub('[^! ]+','', ss)
s_clean

'!!'

In [41]:
## Count number of letters in upper case
##
def perc_symbols(s,pattern):
    try:
        s_clean = re.sub(pattern,'', s)
        return len(s_clean)/len(s)
    except:
        return 0

def count_symbols(s, pattern):
    try:
        s_clean = re.sub(pattern,'', s)
        return len(s_clean)
    except:
        return 0
    
## upper case
##
start = time.clock()
tt = pd.DataFrame([perc_symbols(x,r'[^A-Z ]+') for x in merged['item_description']])
merged['upper_perc_descr'] = tt

tt = pd.DataFrame([perc_symbols(x,r'[^A-Z ]+') for x in merged['name']])
merged['upper_perc_name'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^A-Z ]+') for x in merged['item_description']])
merged['count_upper_descr'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^A-Z ]+') for x in merged['name']])
merged['count_upper_name'] = tt

## exclamation points
tt = pd.DataFrame([perc_symbols(x,r'[^!]+') for x in merged['item_description']])
merged['upper_perc_excl'] = tt

tt = pd.DataFrame([count_symbols(x,r'[^!]+') for x in merged['item_description']])
merged['count_upper_excl'] = tt

print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 0.7774343186215655


In [42]:
## length of description and brand name
start = time.clock()
tt = pd.DataFrame([len(x) for x in merged['item_description']])
merged['len_descr'] = tt

tt = pd.DataFrame([len(x) for x in merged['name']])
merged['len_name'] = tt
print ("Time passed min:",(time.clock()-start)/60)


Time passed min: 0.026048465785513977


In [43]:
merged.loc[1,'item_description']

1    This keyboard is in great condition and works ...
1    25 pcs NEW 7.5"x12" Kraft Bubble Mailers Lined...
Name: item_description, dtype: object

In [44]:
def search_in_str(s,string_val):
    instr = re.findall(string_val,s,flags=re.IGNORECASE)
    if len(instr)>0:
        return 1
    else:
        return 0

def search_in_merged(string_val):
    tt1 = ([search_in_str(x,string_val) for x in merged['item_description']])
    tt2 = ([search_in_str(x,string_val) for x in merged['name']])
    tt =  pd.DataFrame({'tt1':tt1,'tt2':tt2}).max(axis=1)
    return tt

start = time.clock()
merged['a_free'] = search_in_merged('Free')
merged['a_free_ship'] = search_in_merged('Free ship')
merged['a_new'] = search_in_merged('new')
merged['a_sale'] = search_in_merged('sale')
merged['a_gift'] = search_in_merged('gift')
merged['a_nwt'] = search_in_merged('nwt')
merged['a_100perc'] = search_in_merged('100%')
print ("Time passed min:",(time.clock()-start)/60)

Time passed min: 1.1048959297186287


In [45]:
merged.sample(10)[['name','brand_name','item_description','a_free','a_free_ship','a_new','a_sale','a_gift','a_nwt','a_100perc']]

,name,brand_name,item_description,a_free,a_free_ship,a_new,a_sale,a_gift,a_nwt,a_100perc
311357,Npc show bikini,missing,Leather look black bodybuilding competition su...,0,0,0,0,0,0,0
609991,Delias shimmer long sleeve sweater m,dELiA*s,Delias silver shimmer long sleeve sweater size M,0,0,0,0,0,0,0
1086690,Grey lulu long sleeve,Lululemon,"Size 6, no flaws at all, Wore it a couple of t...",0,0,0,0,0,0,0
127216,Victoria secret pink,PINK,Full zip hoodie size small,0,0,0,0,0,0,0
1381053,Sale! Guy Harvey long sleeve t-shirt,Guy Harvey,Guy Harvey t-shirt Size M Long sleeve Like new,0,0,1,1,0,0,0
974099,FSU Seminoles Long Sleeve NWOT,missing,Size medium,0,0,0,0,0,0,0
151629,Coach purse,missing,Beautiful suede detailed purse in great condit...,1,0,0,0,0,0,0
176384,Forever 21 collared sweater,FOREVER 21,Forever 21 collared sweater. Navy and cream st...,1,1,1,0,0,0,0
620048,Mont blanc Pen,missing,Authentic used mont blanc black pen. Cracked t...,0,0,0,0,0,0,0
116165,20g Red Stud Piercing,missing,Crystal Stud Piercing ***This is for ONE stud ...,1,0,1,0,0,0,0


In [46]:
NAME_MIN_DF = 10 ## lower bound for word count
MAX_FEAT_DESCP = 5000

print("Name Encodings")

start = time.clock()

##count = CountVectorizer(min_df=NAME_MIN_DF)
##X_name = count.fit_transform(merged["name"])
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, 
                              ngram_range = (1,3),
                              stop_words = "english")
X_name = count_descp.fit_transform(merged["name"])


print("Category Encoders")
count_category = CountVectorizer()
X_category = count_category.fit_transform(merged["category_name"])
X_gen_cat = count_category.fit_transform(merged["general_cat"])
X_cat1 = count_category.fit_transform(merged["subcat_1"])
X_cat2 = count_category.fit_transform(merged["subcat_2"])
X_cat3 = count_category.fit_transform(merged["subcat_3"])


print("Descp encoders")
count_descp = TfidfVectorizer(max_features = MAX_FEAT_DESCP, 
                              ngram_range = (1,3),
                              stop_words = "english")
X_descp = count_descp.fit_transform(merged["item_description"])

print("Brand encoders")
vect_brand = LabelBinarizer(sparse_output=True)
X_brand = vect_brand.fit_transform(merged["brand_name"])

print("Dummy Encoders")
merged['item_condition_id'] = merged['item_condition_id'].astype('category')
merged['shipping'] = merged['shipping'].astype('category')
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(merged[["item_condition_id", "shipping"]], sparse = True).values)

print ("Time passed min:",(time.clock()-start)/60)

Name Encodings
Category Encoders
Descp encoders
Brand encoders
Dummy Encoders
Time passed min: 5.178648224595313


In [62]:
X = scipy.sparse.hstack((X_dummies, 
                         X_descp,
                         X_brand,
                         X_gen_cat, X_cat1, X_cat2, X_cat3,
                         X_name
                        ,merged[['mis_general_cat','mis_subcat_1','mis_subcat_2','mis_subcat_3','mis_subcat_4','mis_brand_name','mis_item_description']]
                        ,merged[['non_utf_descr','non_utf_name','upper_perc_descr','upper_perc_name']]
                        ##,X_non_utf_uniq
                       ,merged[['a_free','a_free_ship','a_new','a_sale','a_gift','a_nwt','a_100perc']]
                      ,merged[['count_upper_excl']]
                        )).tocsr()
print (X.shape)
## del X_dummies, X_descp, X_brand,X_category,X_name, X_gen_cat, X_cat1, X_cat2, X_cat3
gc.collect()

## 'count_upper_descr','count_upper_name','len_descr','len_name'

(2175894, 13122)


404

In [48]:
## save data
##
scipy.sparse.save_npz('model/x.npz',X)
merged.to_csv('model/merged.csv',encoding='utf-8')

In [155]:
## load data
##

X = scipy.sparse.load_npz('model/x.npz')
merged = pd.read_csv('model/merged.csv')


In [329]:
train_test_split??

In [63]:
y_merged_train = merged['price'][:nrow_train]
y_merged_train = np.log1p(y_merged_train)
X_merged_train = X[:nrow_train]

X_train, X_valid, y_train, y_valid = train_test_split(X_merged_train, y_merged_train, test_size=0.2, random_state =102)
k_fold = KFold(n_splits=5,random_state=101)


print (X_train.shape, len(y_train))
print (X_valid.shape, len(y_valid))


(1186028, 13122) 1186028
(296507, 13122) 296507


In [60]:
## https://www.kaggle.com/apapiu/ridge-script
##

## scorer_func = make_scorer(rmsle,greater_is_better=False)
## scorer_func = make_scorer(r2_score,greater_is_better=True)
scorer_func = make_scorer(mean_squared_error,greater_is_better=False)
## scorer_func = make_scorer(mean_squared_log_error,greater_is_better=False)

mm = Ridge(solver = "lsqr", normalize=False, fit_intercept=False,random_state=120)

scores = cross_val_score(mm, X_train, y_train, 
                         cv=k_fold, n_jobs=-1,scoring = scorer_func)

scores = np.sqrt(-scores)
print (np.mean(scores),scores)

## test on validation
mm.fit(X_train, y_train)
y_pred = mm.predict(X_valid)
y_pred[y_pred<1.386]=1.386

print ("RMSLE: ",rmsle(np.expm1(y_valid),np.expm1(y_pred)))
print ("RMSE: ",mean_squared_error(y_valid,y_pred))
print ("R2: ",r2_score(y_valid,y_pred))

0.494907664851 [ 0.49453879  0.49653881  0.49382434  0.49472717  0.49490921]
RMSLE:  0.49326124138
RMSE:  0.243306652248
R2:  0.566425516024


In [ ]:
0.518020633601 [ 0.52231744  0.5173286   0.52197693  0.51487274  0.51360746]
RMSLE:  0.51304258507
RMSE:  0.263212694095
R2:  0.521618753148

In [65]:
params2 = {
        'learning_rate': 0.7,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 120,
        'verbosity': -1,
        'metric': 'RMSE',
        'data_random_seed': 2,
        'bagging_fraction': 1,
        'nthread': 4
}

start = time.clock()
d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid)


watchlist2 = [d_valid]    
lgb_model = lgb.train(params2, train_set=d_train, num_boost_round=8000, valid_sets=watchlist2, \
    early_stopping_rounds=250, verbose_eval=True)     

y_pred = lgb_model.predict(X_valid)
y_pred[y_pred<1.386]=1.386

print ("RMSLE: ",rmsle(np.expm1(y_valid),np.expm1(y_pred)))
print ("RMSE: ",mean_squared_error(y_valid,y_pred))
print ("R2: ",r2_score(y_valid,y_pred))

print ("Time passed min:",(time.clock()-start)/60)

[1]	valid_0's rmse: 0.703835
Training until validation scores don't improve for 250 rounds.
[2]	valid_0's rmse: 0.685523
[3]	valid_0's rmse: 0.674474
[4]	valid_0's rmse: 0.663869
[5]	valid_0's rmse: 0.655531
[6]	valid_0's rmse: 0.649037
[7]	valid_0's rmse: 0.642634
[8]	valid_0's rmse: 0.637848
[9]	valid_0's rmse: 0.63338
[10]	valid_0's rmse: 0.629714
[11]	valid_0's rmse: 0.626386
[12]	valid_0's rmse: 0.623449
[13]	valid_0's rmse: 0.620662
[14]	valid_0's rmse: 0.617435
[15]	valid_0's rmse: 0.614832
[16]	valid_0's rmse: 0.612596
[17]	valid_0's rmse: 0.610313
[18]	valid_0's rmse: 0.608196
[19]	valid_0's rmse: 0.606266
[20]	valid_0's rmse: 0.604083
[21]	valid_0's rmse: 0.602335
[22]	valid_0's rmse: 0.600808
[23]	valid_0's rmse: 0.599085
[24]	valid_0's rmse: 0.597693
[25]	valid_0's rmse: 0.596076
[26]	valid_0's rmse: 0.594791
[27]	valid_0's rmse: 0.593108
[28]	valid_0's rmse: 0.591284
[29]	valid_0's rmse: 0.590001
[30]	valid_0's rmse: 0.588624
[31]	valid_0's rmse: 0.587638
[32]	valid_0's rm

[269]	valid_0's rmse: 0.515265
[270]	valid_0's rmse: 0.515165
[271]	valid_0's rmse: 0.515062
[272]	valid_0's rmse: 0.514991
[273]	valid_0's rmse: 0.514925
[274]	valid_0's rmse: 0.514819
[275]	valid_0's rmse: 0.514637
[276]	valid_0's rmse: 0.514511
[277]	valid_0's rmse: 0.51436
[278]	valid_0's rmse: 0.514288
[279]	valid_0's rmse: 0.514186
[280]	valid_0's rmse: 0.51412
[281]	valid_0's rmse: 0.514052
[282]	valid_0's rmse: 0.51397
[283]	valid_0's rmse: 0.513899
[284]	valid_0's rmse: 0.513757
[285]	valid_0's rmse: 0.513636
[286]	valid_0's rmse: 0.513548
[287]	valid_0's rmse: 0.513495
[288]	valid_0's rmse: 0.513419
[289]	valid_0's rmse: 0.51334
[290]	valid_0's rmse: 0.51326
[291]	valid_0's rmse: 0.513136
[292]	valid_0's rmse: 0.513049
[293]	valid_0's rmse: 0.512652
[294]	valid_0's rmse: 0.510759
[295]	valid_0's rmse: 0.510498
[296]	valid_0's rmse: 0.510356
[297]	valid_0's rmse: 0.510215
[298]	valid_0's rmse: 0.510096
[299]	valid_0's rmse: 0.509965
[300]	valid_0's rmse: 0.509908
[301]	valid_0

[536]	valid_0's rmse: 0.495425
[537]	valid_0's rmse: 0.495404
[538]	valid_0's rmse: 0.495367
[539]	valid_0's rmse: 0.495335
[540]	valid_0's rmse: 0.495304
[541]	valid_0's rmse: 0.495267
[542]	valid_0's rmse: 0.495242
[543]	valid_0's rmse: 0.495217
[544]	valid_0's rmse: 0.495157
[545]	valid_0's rmse: 0.495131
[546]	valid_0's rmse: 0.495087
[547]	valid_0's rmse: 0.495036
[548]	valid_0's rmse: 0.49502
[549]	valid_0's rmse: 0.495001
[550]	valid_0's rmse: 0.494966
[551]	valid_0's rmse: 0.494932
[552]	valid_0's rmse: 0.494906
[553]	valid_0's rmse: 0.494829
[554]	valid_0's rmse: 0.494792
[555]	valid_0's rmse: 0.494765
[556]	valid_0's rmse: 0.494738
[557]	valid_0's rmse: 0.494694
[558]	valid_0's rmse: 0.494667
[559]	valid_0's rmse: 0.494625
[560]	valid_0's rmse: 0.494578
[561]	valid_0's rmse: 0.494513
[562]	valid_0's rmse: 0.494402
[563]	valid_0's rmse: 0.494181
[564]	valid_0's rmse: 0.494144
[565]	valid_0's rmse: 0.494097
[566]	valid_0's rmse: 0.494064
[567]	valid_0's rmse: 0.494029
[568]	val

[802]	valid_0's rmse: 0.486124
[803]	valid_0's rmse: 0.486102
[804]	valid_0's rmse: 0.486073
[805]	valid_0's rmse: 0.486076
[806]	valid_0's rmse: 0.486058
[807]	valid_0's rmse: 0.486044
[808]	valid_0's rmse: 0.48603
[809]	valid_0's rmse: 0.48601
[810]	valid_0's rmse: 0.485981
[811]	valid_0's rmse: 0.485969
[812]	valid_0's rmse: 0.485941
[813]	valid_0's rmse: 0.485915
[814]	valid_0's rmse: 0.485891
[815]	valid_0's rmse: 0.485877
[816]	valid_0's rmse: 0.485854
[817]	valid_0's rmse: 0.48584
[818]	valid_0's rmse: 0.485833
[819]	valid_0's rmse: 0.485819
[820]	valid_0's rmse: 0.485792
[821]	valid_0's rmse: 0.485789
[822]	valid_0's rmse: 0.485766
[823]	valid_0's rmse: 0.485743
[824]	valid_0's rmse: 0.485719
[825]	valid_0's rmse: 0.485688
[826]	valid_0's rmse: 0.485669
[827]	valid_0's rmse: 0.485646
[828]	valid_0's rmse: 0.48563
[829]	valid_0's rmse: 0.48562
[830]	valid_0's rmse: 0.485614
[831]	valid_0's rmse: 0.485603
[832]	valid_0's rmse: 0.485588
[833]	valid_0's rmse: 0.485576
[834]	valid_0

[1066]	valid_0's rmse: 0.481218
[1067]	valid_0's rmse: 0.481172
[1068]	valid_0's rmse: 0.481156
[1069]	valid_0's rmse: 0.481145
[1070]	valid_0's rmse: 0.48114
[1071]	valid_0's rmse: 0.481124
[1072]	valid_0's rmse: 0.481109
[1073]	valid_0's rmse: 0.481109
[1074]	valid_0's rmse: 0.481101
[1075]	valid_0's rmse: 0.481089
[1076]	valid_0's rmse: 0.481081
[1077]	valid_0's rmse: 0.481075
[1078]	valid_0's rmse: 0.481062
[1079]	valid_0's rmse: 0.481061
[1080]	valid_0's rmse: 0.481061
[1081]	valid_0's rmse: 0.481026
[1082]	valid_0's rmse: 0.48102
[1083]	valid_0's rmse: 0.48101
[1084]	valid_0's rmse: 0.480898
[1085]	valid_0's rmse: 0.480839
[1086]	valid_0's rmse: 0.480828
[1087]	valid_0's rmse: 0.4808
[1088]	valid_0's rmse: 0.480783
[1089]	valid_0's rmse: 0.480778
[1090]	valid_0's rmse: 0.480771
[1091]	valid_0's rmse: 0.480761
[1092]	valid_0's rmse: 0.480752
[1093]	valid_0's rmse: 0.480726
[1094]	valid_0's rmse: 0.480711
[1095]	valid_0's rmse: 0.480708
[1096]	valid_0's rmse: 0.480704
[1097]	valid_

...

[4420]	valid_0's rmse: 0.466793
[4421]	valid_0's rmse: 0.466796
[4422]	valid_0's rmse: 0.466794
[4423]	valid_0's rmse: 0.466795
[4424]	valid_0's rmse: 0.466796
[4425]	valid_0's rmse: 0.466796
[4426]	valid_0's rmse: 0.466786
[4427]	valid_0's rmse: 0.466791
[4428]	valid_0's rmse: 0.466791
[4429]	valid_0's rmse: 0.46678
[4430]	valid_0's rmse: 0.466773
[4431]	valid_0's rmse: 0.466782
[4432]	valid_0's rmse: 0.466788
[4433]	valid_0's rmse: 0.466789
[4434]	valid_0's rmse: 0.466788
[4435]	valid_0's rmse: 0.466792
[4436]	valid_0's rmse: 0.466786
[4437]	valid_0's rmse: 0.466781
[4438]	valid_0's rmse: 0.466786
[4439]	valid_0's rmse: 0.466787
[4440]	valid_0's rmse: 0.466783
[4441]	valid_0's rmse: 0.466783
[4442]	valid_0's rmse: 0.466783
[4443]	valid_0's rmse: 0.466782
[4444]	valid_0's rmse: 0.466787
[4445]	valid_0's rmse: 0.466788
[4446]	valid_0's rmse: 0.466791
[4447]	valid_0's rmse: 0.466793
[4448]	valid_0's rmse: 0.466792
[4449]	valid_0's rmse: 0.466792
[4450]	valid_0's rmse: 0.466795
[4451]	va

[4678]	valid_0's rmse: 0.46666
[4679]	valid_0's rmse: 0.466658
[4680]	valid_0's rmse: 0.466644
[4681]	valid_0's rmse: 0.46664
[4682]	valid_0's rmse: 0.466639
[4683]	valid_0's rmse: 0.466637
[4684]	valid_0's rmse: 0.466634
[4685]	valid_0's rmse: 0.466624
[4686]	valid_0's rmse: 0.466623
[4687]	valid_0's rmse: 0.466617
[4688]	valid_0's rmse: 0.466603
[4689]	valid_0's rmse: 0.466597
[4690]	valid_0's rmse: 0.466587
[4691]	valid_0's rmse: 0.466585
[4692]	valid_0's rmse: 0.466576
[4693]	valid_0's rmse: 0.466581
[4694]	valid_0's rmse: 0.466576
[4695]	valid_0's rmse: 0.46656
[4696]	valid_0's rmse: 0.466562
[4697]	valid_0's rmse: 0.466564
[4698]	valid_0's rmse: 0.466561
[4699]	valid_0's rmse: 0.466568
[4700]	valid_0's rmse: 0.466566
[4701]	valid_0's rmse: 0.466572
[4702]	valid_0's rmse: 0.466578
[4703]	valid_0's rmse: 0.466576
[4704]	valid_0's rmse: 0.466576
[4705]	valid_0's rmse: 0.466577
[4706]	valid_0's rmse: 0.466577
[4707]	valid_0's rmse: 0.466576
[4708]	valid_0's rmse: 0.466575
[4709]	vali

[4936]	valid_0's rmse: 0.466383
[4937]	valid_0's rmse: 0.466382
[4938]	valid_0's rmse: 0.46637
[4939]	valid_0's rmse: 0.466366
[4940]	valid_0's rmse: 0.466371
[4941]	valid_0's rmse: 0.466377
[4942]	valid_0's rmse: 0.466372
[4943]	valid_0's rmse: 0.466379
[4944]	valid_0's rmse: 0.46638
[4945]	valid_0's rmse: 0.466379
[4946]	valid_0's rmse: 0.466381
[4947]	valid_0's rmse: 0.46638
[4948]	valid_0's rmse: 0.466377
[4949]	valid_0's rmse: 0.466378
[4950]	valid_0's rmse: 0.466377
[4951]	valid_0's rmse: 0.466376
[4952]	valid_0's rmse: 0.466379
[4953]	valid_0's rmse: 0.466385
[4954]	valid_0's rmse: 0.466379
[4955]	valid_0's rmse: 0.466377
[4956]	valid_0's rmse: 0.466379
[4957]	valid_0's rmse: 0.46638
[4958]	valid_0's rmse: 0.466379
[4959]	valid_0's rmse: 0.466383
[4960]	valid_0's rmse: 0.466382
[4961]	valid_0's rmse: 0.466384
[4962]	valid_0's rmse: 0.466389
[4963]	valid_0's rmse: 0.466393
[4964]	valid_0's rmse: 0.4664
[4965]	valid_0's rmse: 0.466396
[4966]	valid_0's rmse: 0.466399
[4967]	valid_0

[5194]	valid_0's rmse: 0.466352
[5195]	valid_0's rmse: 0.466353
[5196]	valid_0's rmse: 0.466351
[5197]	valid_0's rmse: 0.466349
[5198]	valid_0's rmse: 0.466356
[5199]	valid_0's rmse: 0.466357
[5200]	valid_0's rmse: 0.466357
[5201]	valid_0's rmse: 0.466353
[5202]	valid_0's rmse: 0.466351
[5203]	valid_0's rmse: 0.466347
[5204]	valid_0's rmse: 0.466347
[5205]	valid_0's rmse: 0.466344
[5206]	valid_0's rmse: 0.466345
[5207]	valid_0's rmse: 0.466344
[5208]	valid_0's rmse: 0.466343
[5209]	valid_0's rmse: 0.466341
[5210]	valid_0's rmse: 0.466345
[5211]	valid_0's rmse: 0.466343
[5212]	valid_0's rmse: 0.466342
[5213]	valid_0's rmse: 0.46634
[5214]	valid_0's rmse: 0.466342
[5215]	valid_0's rmse: 0.46634
[5216]	valid_0's rmse: 0.466343
[5217]	valid_0's rmse: 0.466346
[5218]	valid_0's rmse: 0.466347
[5219]	valid_0's rmse: 0.466343
[5220]	valid_0's rmse: 0.466344
[5221]	valid_0's rmse: 0.46635
[5222]	valid_0's rmse: 0.466349
[5223]	valid_0's rmse: 0.46635
[5224]	valid_0's rmse: 0.466351
[5225]	valid

[5452]	valid_0's rmse: 0.466415
[5453]	valid_0's rmse: 0.466411
[5454]	valid_0's rmse: 0.466413
[5455]	valid_0's rmse: 0.466409
[5456]	valid_0's rmse: 0.466405
[5457]	valid_0's rmse: 0.466407
[5458]	valid_0's rmse: 0.46641
[5459]	valid_0's rmse: 0.466415
[5460]	valid_0's rmse: 0.466416
[5461]	valid_0's rmse: 0.466415
[5462]	valid_0's rmse: 0.466419
[5463]	valid_0's rmse: 0.466419
Early stopping, best iteration is:
[5213]	valid_0's rmse: 0.46634
RMSLE:  0.466236276326
RMSE:  0.217376265363
R2:  0.612633681765
Time passed min: 12.429295251109261


In [ ]:
X_test = X[nrow_train:]

y_pred = lgb_model.predict(X_test)

y_pred[y_pred<1.386]=1.386
y_pred = np.expm1(y_pred)

sub = pd.DataFrame()
sub['test_id'] = merged['test_id'][nrow_train:].astype(int)
sub['price'] = y_pred
sub.to_csv('result.csv', index=False)
print ("fini")


In [ ]:
## https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling
##